# 1


# 2
假设现在有一个4x4的具有两个通道的特征如下所示。
```
f = [[[ 1  2  3  4]
      [ 8  7  6  5]
      [ 9 10 11 12]
      [16 15 14 13]]

     [[29 30 31 32]
      [28 27 26 25]
      [21 22 23 24]
      [20 19 18 17]]]
```
对这个图像采用，如下的模板进行模板运算。
```
h = [[[[-1  0  1]
       [-1  0  1]
       [-1  0  1]]
      [[-1 -1 -1]
       [ 0  0  0]
       [ 1  1  1]]]

     [[[ 1  0  0]
       [ 0  1  0]
       [ 0  0  1]]
      [[ 0  0  1]
       [ 0  1  0]
       [ 1  0  0]]]]
```

In [21]:
import numpy as np

# f[c_in, x, y]
f = np.asarray([[[1, 2, 3, 4],
                 [8, 7, 6, 5],
                 [9, 10, 11, 12],
                 [16, 15, 14, 13]],
                [[29, 30, 31, 32],
                 [28, 27, 26, 25],
                 [21, 22, 23, 24],
                 [20, 19, 18, 17]]])

h = np.empty([2, 2, 3, 3])
h[0, 0, :, :] = [[-1, 0, 1],
                 [-1, 0, 1],
                 [-1, 0, 1]]
h[0, 1, :, :] = [[-1, -1, -1],
                 [0, 0, 0],
                 [1, 1, 1]]
h[1, 0, :, :] = [[1, 0, 0],
                 [0, 1, 0],
                 [0, 0, 1]]
h[1, 1, :, :] = [[0, 0, 1],
                 [0, 1, 0],
                 [1, 0, 0]]

模板运算采用 Valid 输出尺寸，请问：

1）输出记为 conv1 ，请问 conv1 是多少？

In [33]:
from scipy.signal import convolve2d

_, M, N = f.shape
# conv1[c_out, c_in, x, y]
conv1 = np.empty([2, 2, M - 2, N - 2])
for c_out in range(2):
    for c_in in range(2):
        conv1[c_out, c_in, :, :] = convolve2d(f[c_in, :, :], np.rot90(h[c_out, c_in, :, :], 2), mode="valid")

print("conv1 = \n", conv1)

# print(convolve2d(f[0,:,:],np.rot90(h[0,0,:,:],2),"valid"))

conv1 = 
 [[[[  2.   2.]
   [ -2.  -2.]]

  [[-24. -24.]
   [-24. -24.]]]


 [[[ 19.  20.]
   [ 32.  31.]]

  [[ 79.  80.]
   [ 68.  67.]]]]


(2) 如果采用 ReLU 对这个输出进行激活，记为 relu1 ，请问激活后 relu1 的值是多
少？

In [38]:
def relu(np_vect):
    if np_vect >= 0:
        return np_vect
    else:
        return 0


rectification = np.vectorize(relu)
relu1 = rectification(conv1)

print(relu1)

[[[[ 2.  2.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[19. 20.]
   [32. 31.]]

  [[79. 80.]
   [68. 67.]]]]
400.0


3）如果将输出拉成一列，采用全连接网络，输出节点个数为 5，假设全连接所有权重都设置为 1/10，输出记为fc1 ，请问输出是多少？

In [35]:
fc1 = np.zeros([5])
weight = 0.1

for i in range(5):
    for j in relu1.flat:
        fc1[i] += weight * j
print(fc1)

[40. 40. 40. 40. 40.]


4）假设采用 softmax 对这个 5 个节点的输出进行，概率值记为p=[p1,p2,p3,p4,p5]，
请问p是多少？

In [36]:
p = np.asarray([1 / (1 + np.exp(-x)) for x in fc1])

print(p)

[1.0, 1.0, 1.0, 1.0, 1.0]


5) 如果采用交叉熵对概率进行约束，如下所示

$$L=\sum_{i=1}^{5}-y_{i} \log p_{i}$$

如果$y_{1}=0, y_{2}=0, y_{3}=1, y_{4}=0, y_{5}=0$，请问损失函数是多少？

In [42]:
y = np.asarray([0, 0, 1, 0, 0])
print("log(p) = ", np.log(p))
L = np.sum(y * np.log(p))
print("L = ", L)

log(p) =  [0. 0. 0. 0. 0.]
L =  0.0


6) 请问$\frac{\partial L}{\partial p}, \frac{\partial L}{\partial \mathrm{fc}_{1}}, \frac{\partial L}{\partial \mathrm{relu}_{1}}, \frac{\partial L}{\partial \operatorname{conv}_{1}}$分别是多少？

7) 如果把全连接的权重记为$W$，请问$\frac{\partial L}{\partial W}$是多少？

8) 请问$\frac{\partial L}{\partial h}$是多少？